# Automatización de Contratos

A través del siguiente notebook se desarrolla la clase `ContractExtractor` que transforma un conjunto de contratos en PDF en una base de datos en formato excel con los campos:
1. Contract Amount
2. Effective Date
3. Term
4. Signature Date
5. Service Description
6. Service Cancellation Clause

### Funcionamiento
1. En primer lugar, cada documento es validado. Se verifica que se encuentra en el formato correcto (PDF) y que tiene mas de 200 caracteres. El resultado del proceso de validacion se muestra con las columnas:
- **valid**: Un booleano que refleja la validez del documento PDF.
- **valid_note**: Notas que explican el error de validez.

2. Luego, se extrae programaticamente cada campo, junto a otras 4 columnas que describen su extraccion:
- **[Campo]**: El valor extraido
- **[Campo] Found**: Un booleano que representa si el valor fue encontrado en el documento
- **[Campo] Origin**: Una cadena de texto que lista todos los parrafos que fueron utilizados para extraer el campo (Contexto).
- **[Campo] Confidence**: El nivel de confianza en la certeza de la extraccion. Puede ser 'low', 'medium' y 'high'.
- **[Campo] Notes**: Notas que explican el proceso de extraccion.

### Paralelismo

Se implementó la funcion `parallel_apply` para permitir procesar multiples contratos al mismo tiempo y reducir significativamente el tiempo de carga.

In [ ]:
# Para ejecutar en caso de no haber instalado los requerimientos en un entorno virtual previamente
%%capture
!pip install python-dotenv
!pip install promptlayer
!pip install pandarallel
!pip install PyPDF2
!pip install anthropic
!pip install openpyxl
!pip install Jinja2

In [1]:
import os
import logging
import PyPDF2
import time
import re
import json
import jinja2
import base64
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import anthropic
from promptlayer import PromptLayer

from dotenv import load_dotenv
load_dotenv()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


True

In [8]:
class ContractExtractor():
  """Extrae informacion de contratos en PDF a una base de datos en excel"""

  def __init__(self, contracts_dir: str, save_path: str, checkpoint_path: str):
    """
    Inicializa la clase ProcessContracts.

    Args:
      contracts_dir: La ruta al directorio que contiene los archivos PDF de contratos.
      save_path: La ruta donde se guardará la base de datos en formato Excel.
      checkpoint_path: La ruta donde se guardará el ultimo estado de la DB.

    Raises:
      AssertionError: Si el directorio de contratos no existe.
    """
    self.logger = logging.getLogger(__name__)
    self.logger.propagate = False
    logging.basicConfig(
       filename="app.log",
       filemode='a',
       level=logging.INFO
    )

    assert os.path.isdir(contracts_dir), "Contracts directory not found."

    self.contracts_dir: str = contracts_dir
    self.contracts_paths: list[str] = \
     [f for f in os.listdir(contracts_dir) if os.path.isfile(os.path.join(contracts_dir, f))]

    assert len(self.contracts_paths) > 0, "Contracts directory is empty."

    self.SAVE_PATH: str = save_path
    self.CHECKPOINT_PATH: str = checkpoint_path

    promptlayer_client = PromptLayer()
    raw_template = promptlayer_client.templates.get("Extractor de Campo")['prompt_template']['messages'][0]['content'][0]['text']
    self.prompt_template = jinja2.Template(raw_template)

    self.FIELDS_METADATA = {
        "Contract Amount": {
            "field_type": "float",
            "field_description": "Total contract amount",
        },
        "Effective Date": {
            "field_type": "string",
            "field_description": "Date when the contract is effective, in the format YYYY-MM-DD",
        },
        "Term": {
            "field_type": "string",
            "field_description": "End date of the contract in YYYY-MM-DD format",
        },
        "Signature Date": {
            "field_type": "string",
            "field_description": "Date when the contract was signed, in the format YYYY-MM-DD",
        },
        "Service Description": {
            "field_type": "string",
            "field_description": "Detailed description of the service being provided",
        },
        "Service Cancellation Clause": {
            "field_type": "string",
            "field_description": "Clause describing the terms for service cancellation",
        },
    }
    self.FIELDS_LIST = list(self.FIELDS_METADATA.keys())

    self.contracts_df = self._init_dataframe()
    self.contracts_df['contract_path'] = self.contracts_paths

    self.checkpoint_contracts_df = self.contracts_df.copy()

  def checkpoint(self):
    """Guarda el estado actual de la base de datos en el archivo de checkpoint"""
    self.checkpoint_contracts_df.to_excel(self.CHECKPOINT_PATH, index=False)

  def save_df(self):
    """Guarda la base de datos en el archivo de salida"""
    self.extracted_contracts_df.to_excel(self.SAVE_PATH, index=False)

  def _init_dataframe(self) -> pd.DataFrame:
    """
    Inicializa un DataFrame vacío para almacenar los resultados.

    Returns:
        Un DataFrame vacío.
    """
    contracts_df = {"contract_path": pd.Series(dtype="string")}
    contracts_df["valid"] = pd.Series(dtype="bool")
    contracts_df["valid_note"] = pd.Series(dtype="string")

    for field in self.FIELDS_LIST:
      contracts_df[field] = pd.Series(dtype=self.FIELDS_METADATA[field]['field_type'])
      contracts_df[field + " Found"] = pd.Series(dtype="bool")
      contracts_df[field + " Origin"] = pd.Series(dtype="string")
      contracts_df[field + " Confidence"] = pd.Series(dtype=pd.CategoricalDtype(categories=["low", "medium", "high"]))
      contracts_df[field + " Notes"] = pd.Series(dtype="string")

    contracts_df = pd.DataFrame(contracts_df)

    return contracts_df

  def _validate_contract(self, pdf_path: str) -> tuple[bool, str]:
      """
      Valida un solo contrato PDF.

      Args:
          pdf_path: La ruta al archivo PDF.

      Returns:
          Una tupla que contiene un valor booleano indicando la validez y una cadena descriptiva.
      """
      try:
          with open(os.path.join(self.contracts_dir, pdf_path), 'rb') as pdf_file:
              pdf_reader = PyPDF2.PdfReader(pdf_file)
              text = ""
              for page in pdf_reader.pages:
                  text += page.extract_text()
              if len(text) >= 200:
                  return True, "Valid contract"
              else:
                  return False, "Contract too short"

      except PyPDF2.errors.PdfReadError:
          return False, "Corrupted PDF"
      except FileNotFoundError:
          return False, "PDF not found"

  def _get_prompt(self, field: str, field_type: str, field_description: str) -> str:
    """
    Genera un prompt para extraer un campo específico de un contrato.

    Args:
        field: El nombre del campo a extraer.
        field_type: El tipo de dato del campo.
        field_description: Una descripción detallada del campo.

    Returns:
        Un string que representa el prompt para extraer el campo.
    """
    return self.prompt_template.render(**{
        "FIELD": field,
        "FIELD_TYPE": field_type,
        "FIELD_DESCRIPTION": field_description
    })

  def _parse_output(self, text) -> dict:
    """
    Convierte el texto crudo de salida a JSON

    Args:
      text: El texto crudo de salida

    Returns:
      Un diccionario con la informacion extraida
    """
    # Regular expression to match the first <output> tag and its content
    pattern = r"<output\b[^>]*>(.*?)</output>"

    # Search for the first match
    match = re.search(pattern, text, flags=re.DOTALL | re.IGNORECASE)

    # Return the matched content or None
    raw_json_output =  match.group(1).strip() if match else None

    cleaned_json_output = raw_json_output.replace("\n", "").replace("\r", "")

    json_output = json.loads(cleaned_json_output)

    return json_output

  def _extract_col(self, pdf_path: str, col_name: str) -> tuple[dict, str]:
    """
    Extrae un campo específico de un contrato PDF.

    Args:
        pdf_path: La ruta al archivo PDF.
        col_name: El nombre del campo a extraer.

    Returns:
        Un diccionario con la información extraída y el origen de la información.
    """
    self.logger.debug(f"Extracting field {col_name} from contract {pdf_path}")
    start_time = time.time()

    field_metadata = self.FIELDS_METADATA[col_name]
    prompt = self._get_prompt(
        field=col_name,
        field_type=field_metadata['field_type'],
        field_description=field_metadata['field_description']
    )

    with open(os.path.join(self.contracts_dir, pdf_path), "rb") as f:
      pdf_data = base64.standard_b64encode(f.read()).decode("utf-8")

    try:
      client = anthropic.Anthropic()

      response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        system=prompt,
        messages=[
          {
            "role": "user",
            "content": [
              {
                "type": "document",
                "source": {
                  "type": "base64",
                  "media_type": "application/pdf",
                  "data": pdf_data
                },
                "title": pdf_path,
                "citations": {
                  "enabled": True
                }
              },
            ]
          }
        ]
      )

    except anthropic.RateLimitError as e:
      retry_after = e.response.headers.get("retry-after", "unknown")
      self.logger.error(f"Anthropic API Rate Limit Error while extracting {col_name} from contract \'{pdf_path}\'. Retrying in {retry_after}")
      time.sleep(int(retry_after))
      return self._extract_col(pdf_path, col_name)

    except Exception as e:
      self.logger.error(f"Error extracting field {col_name} from contract {pdf_path}: {e}")
      return None

    raw_output = ''.join([textblock.text for textblock in response.content])
    json_output = self._parse_output(raw_output)

    citations_list = [textblock.citations for textblock in response.content \
                      if textblock.citations != None]
    citations_list = [x for xs in citations_list for x in xs]

    origen = ''
    for citation in citations_list:
      origen += f"Pagina de Inicio: {citation.start_page_number}\nPagina de Fin: {citation.end_page_number}\nCita: {citation.cited_text}\n\n"

    json_output['origen'] = origen

    end_time = time.time()
    gen_duration = round((end_time - start_time), 2)
    self.logger.debug(f"Field {col_name} extracted from contract {pdf_path} in {gen_duration} seconds")

    return json_output

  def _save_row(self, row):
    """
    Guarda una fila en el DataFrame.

    Args:
        row: Una fila del DataFrame.
    """
    row_idx = row.name
    self.logger.debug(f"Saving row {row_idx}...")

    self.checkpoint_contracts_df.loc[row_idx] = row
    self.checkpoint()

  def _extract_row_cols(self, row: pd.Series) -> pd.Series:
    """
    Extrae los 6 campos requeridos a partir de un contrato PDF.

    Args:
        row: Una fila del DataFrame.

    Returns:
        Una nueva fila con los campos extraídos.
    """
    PATH = row['contract_path']
    self.logger.debug(f"Extracting fields from contract {PATH}")

    valid, valid_note = self._validate_contract(PATH)
    row['valid'] = valid
    row['valid_note'] = valid_note

    if valid:
      for field in self.FIELDS_LIST:
        extracted_field = self._extract_col(PATH, field)

        if extracted_field is None:
          continue

        row[field] = extracted_field["value"]
        row[field + " Found"] = extracted_field["found"]
        row[field + " Origin"] = extracted_field["origen"]
        row[field + " Confidence"] = extracted_field["confidence"]
        row[field + " Notes"] = extracted_field["notes"]

    else:
      self.logger.error(f"Contract {PATH} is not valid: {valid_note}")

    self._save_row(row)
    return row

  def extract(self):
    """Extracts 6 columns from every contract in the contracts folder sequentially"""
    self.extracted_contracts_df = self.contracts_df.apply(self._extract_row_cols, axis=1)
    self.save_df()

  def parallel_extract(self):
    """Extracts 6 columns from every contract in the contracts folder in parallel"""
    self.extracted_contracts_df = self.contracts_df.parallel_apply(self._extract_row_cols, axis=1)
    self.save_df()

In [ ]:
ext = ContractExtractor("contratos", "output/output.xlsx", "output/output_checkpoint.xlsx")
ext.extract()

Undefined variable in message index 0: 'FIELD' is undefined
Contract Too Short Contract.pdf is not valid: Contract too short
Contract Corrupted Contract.pdf is not valid: Corrupted PDF
Error extracting field Contract Amount from contract Contract Example.pdf: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Error extracting field Effective Date from contract Contract Example.pdf: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Error extracting field Term from contract Contract Example.pdf: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & 

In [ ]:
output = pd.read_excel("output.xlsx")

In [ ]:
output.head()

,contract_path,valid,valid_note,Contract Amount,Contract Amount Found,Contract Amount Origin,Contract Amount Confidence,Contract Amount Notes,Effective Date,Effective Date Found,...,Service Description,Service Description Found,Service Description Origin,Service Description Confidence,Service Description Notes,Service Cancellation Clause,Service Cancellation Clause Found,Service Cancellation Clause Origin,Service Cancellation Clause Confidence,Service Cancellation Clause Notes
0,Corrupted Contract.pdf,False,Corrupted PDF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Too Short Contract.pdf,False,Contract too short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Contract Example.pdf,True,Valid contract,72000.0,1.0,Pagina de Inicio: 2\nPagina de Fin: 3\nCita: F...,high,The contract amount is calculated from the sta...,2021-07-26,1.0,...,PROVIDER will support with ongoing project man...,1.0,Pagina de Inicio: 1\nPagina de Fin: 2\nCita: S...,high,The service description is clearly stated in t...,NaN,0.0,Pagina de Inicio: 3\nPagina de Fin: 4\nCita: S...,medium,The document references a main Services Agreem...
